In [1]:
# parse the original result: 
# start, end, length, number of reads
# filename='test_with_del_400_1_formated.fa'
original_locations = [[643169,644858,1690,2],
[13480846,13512879,32034,8],
[8039367,8047932,8566,3],
[18704644,18881398,176755,7],
[23607182,23613596,6415,4],
[28465981,28599394,133414,6],
[30804149,30979616,175468,6],
[35940180,36199656,259477,4],
[44149293,44465270,315978,4],
[46379887,46674734,294848,3]]

In [2]:
break_points = []

for i in range(len(original_locations)):
    break_points.append(original_locations[i][0])
    break_points.append(original_locations[i][1])
    
break_points = sorted(break_points)

print(break_points)

[643169, 644858, 8039367, 8047932, 13480846, 13512879, 18704644, 18881398, 23607182, 23613596, 28465981, 28599394, 30804149, 30979616, 35940180, 36199656, 44149293, 44465270, 46379887, 46674734]


In [3]:
# filename='test_with_del_400_1_formated.fa', Histogram Size: 1000
CNV_pytor_calls = [["duplication","chr22",8040001,8048000,8000,2.9704977216326194,0.0],
["duplication","chr22",13481001,13513000,32000,8.049257412365229,0.0],
["duplication","chr22",18658001,18683000,25000,1.60915345766002,0.00041953754054184885],
["duplication","chr22",18705001,18881000,176000,5.320488693074513,0.00011267605633802817],
["duplication","chr22",20326001,20368000,42000,1.6548820347955073,0.00046664862291991374],
["duplication","chr22",20610001,20655000,45000,2.080531788919076,0.0001596490497410909],
["duplication","chr22",21466001,21505000,39000,1.7381756742881092,0.00019154336062826222],
["duplication","chr22",21522001,21559000,37000,1.6756902740988295,0.0018435303606406269],
["duplication","chr22",21561001,21686000,125000,1.9209141244430774,0.0005301137039801803],
["duplication","chr22",23608001,23613000,5000,3.976525960397803,0.0],
["duplication","chr22",28466001,28599000,133000,5.963115601600853,4.425141358682291e-05],
["duplication","chr22",30804001,30989000,185000,5.76507271807186,1.0983713593565985e-05],
["duplication","chr22",35940001,36200000,260000,4.001903475828304,2.999651290537475e-05],
["duplication","chr22",44149001,44465000,316000,4.00183080907843,2.4692132829270878e-05],
["duplication","chr22",46380001,46675000,295000,3.000748484978554,2.4998290558072133e-05]]

len(CNV_pytor_calls)

15

In [4]:
for call in CNV_pytor_calls:
    call.pop(1) # removed chr22 part, not important for analysis
    call.pop(5) # removed p-value part, not important for analysis
    call[4] = round(call[4]) # rounded the copy number to an integer


In [5]:
for i, call in enumerate(CNV_pytor_calls):
    # print(i) # i prints 30, so newly added cnv_pytor_calls are added to the end of the list
    type_, start, end, length, copy_number = call

    for breakpoint in break_points:
        if start < breakpoint < end:
            # print("breakpoint:", breakpoint, "start:", start, "end:", end) 
            CNV_pytor_calls.append([type_, start, breakpoint, breakpoint-start + 1, copy_number])
            CNV_pytor_calls.append([type_, breakpoint + 1, end, end-breakpoint, copy_number])
            CNV_pytor_calls[i] = None
            break

In [6]:
# remove None from the list
CNV_pytor_calls = [call for call in CNV_pytor_calls if call is not None]
CNV_pytor_calls = sorted(CNV_pytor_calls, key=lambda x: x[1])
CNV_pytor_calls

[['duplication', 8040001, 8047932, 7932, 3],
 ['duplication', 8047933, 8048000, 68, 3],
 ['duplication', 13481001, 13512879, 31879, 8],
 ['duplication', 13512880, 13513000, 121, 8],
 ['duplication', 18658001, 18683000, 25000, 2],
 ['duplication', 18705001, 18881000, 176000, 5],
 ['duplication', 20326001, 20368000, 42000, 2],
 ['duplication', 20610001, 20655000, 45000, 2],
 ['duplication', 21466001, 21505000, 39000, 2],
 ['duplication', 21522001, 21559000, 37000, 2],
 ['duplication', 21561001, 21686000, 125000, 2],
 ['duplication', 23608001, 23613000, 5000, 4],
 ['duplication', 28466001, 28599000, 133000, 6],
 ['duplication', 30804001, 30804149, 149, 6],
 ['duplication', 30804150, 30979616, 175467, 6],
 ['duplication', 30979617, 30989000, 9384, 6],
 ['duplication', 35940001, 35940180, 180, 4],
 ['duplication', 35940181, 36199656, 259476, 4],
 ['duplication', 36199657, 36200000, 344, 4],
 ['duplication', 44149001, 44149293, 293, 4],
 ['duplication', 44149294, 44465000, 315707, 4],
 ['dup

In [7]:
true_positive = 0
false_positive = 0
ground_truth = 0

In [8]:
# find the ground truth
for original_call in original_locations:
    start, end, length, number_of_reads = original_call
    ground_truth += length*number_of_reads

In [9]:
for cnvpytor_call in CNV_pytor_calls:
    cnvpytor_type, cnvpytor_start, cnvpytor_end, cnvpytor_length, cnvpytor_copy_number = cnvpytor_call
    has_overlapped = False
    for original_call in original_locations:
        original_start, original_end, original_length, original_copy_number = original_call
        # find the overlap between two calls
        overlap = min(cnvpytor_end, original_end) - max(cnvpytor_start, original_start) + 1

        if overlap < 0:
            continue

        has_overlapped = True

        true_positive += overlap * min(original_copy_number, cnvpytor_copy_number)

        if cnvpytor_copy_number > original_copy_number:
            false_positive += cnvpytor_length * (cnvpytor_copy_number - original_copy_number)

    if not has_overlapped:
        false_positive += cnvpytor_length * (cnvpytor_copy_number-1)
        

In [10]:
false_negative = ground_truth - true_positive

In [11]:
print("true_positive:", true_positive)
print("false_positive:", false_positive)
print("false_negative:", false_negative)
print("ground_truth:", ground_truth)

true_positive: 6214578
false_positive: 626000
false_negative: 373373
ground_truth: 6587951


In [12]:
true_positive / ground_truth

0.9433248668667997

In [13]:
false_positive / ground_truth

0.09502195751000576

In [14]:
false_negative / ground_truth

0.056675133133200294